# Datasetin luonti analyysia varten
[Cognition.run](https://www.cognition.run) tallentaa jokaisen kokeen aikana tapahtuneen tapahtuman omalle rivilleen. Suurin osa tapahtumista eivät ole tarpeellisia analyysin kannalta, joten ne on siivottava datasetistä pois. Datasetti tulee myös muokata sellaiseen muotoon, että siitä on helpompi tehdä analyyseja.

31 yksi henkilöä ilmoittautui osallistujaksi kokeeseen, joista 30 suoritti kokeen (ID 24 ei osallistunut). Yksi osallistujista osallistui kokeeseen englannin kielellä, joten hänen dataansa ei käytetä analyysissa (ID 7). Yksi osallistujista kysyi kokeen suoritettuaan "missä vaiheessa se puhelimeen liittyvä osuus on?". Hänen dataansa ei myöskään käytetä analyysissa, koska selvästi tiesi kokeen tavoitteista liikaa ennen kokeeseen osallistumista (ID 1). Otoksen koko on siis yhteensä **28**.

Ladataan ensin cognition.run sivustolta ladattu raaka data. Suomen ja englannin kieliset tehtävät olivat palvelussa erillään, joten tässä datasetissä on vain suomenkielellä suoritetut tehtävät.

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np

# Ospan aineisto

In [2]:
df=pd.read_csv('Raw data/ospan-suomi.csv', delimiter=',' )

## Puuttuva osallistuja ospan datassa

In [3]:
print('Uniikit: ', df['subjectId'].unique())
print('Yhteensä: ',df['subjectId'].unique().size)

Uniikit:  [ 1  3 10 11  8 12 14 13  9  2 15 16  6 23 22 17 21  4 18 29 19 20 28 25
 27 26 30  5]
Yhteensä:  28


Osallistuja numeroita näyttäisi olevan yksi liian vähän, sillä suomen kielellä kokeen suoritti 29 henkilöä. 

SubjectId 31 on virheellisesti kirjattu subjectId 30 alle.

In [4]:
print('SubjectId = 29')
print(df['recall'][df['subjectId'] == 29].info())
print('\nSubjectId = 30')
print(df['recall'][df['subjectId'] == 30].info())
print('\nSubjectId = 31')
print(df['recall'][df['subjectId'] == 31].info())

SubjectId = 29
<class 'pandas.core.series.Series'>
Int64Index: 212 entries, 3710 to 5299
Series name: recall
Non-Null Count  Dtype 
--------------  ----- 
12 non-null     object
dtypes: object(1)
memory usage: 3.3+ KB
None

SubjectId = 30
<class 'pandas.core.series.Series'>
Int64Index: 424 entries, 5300 to 5935
Series name: recall
Non-Null Count  Dtype 
--------------  ----- 
24 non-null     object
dtypes: object(1)
memory usage: 6.6+ KB
None

SubjectId = 31
<class 'pandas.core.series.Series'>
Int64Index: 0 entries
Series name: recall
Non-Null Count  Dtype 
--------------  ----- 
0 non-null      object
dtypes: object(1)
memory usage: 0.0+ bytes
None


Kokeen toteutuksen kirjanpidosta tiedän, että subjectId 31 suoritti kokeen kaikkein viimeisenä ennen subjectId 30. Etsitään mistä kellon ajasta alkaen subjectId 31 suoritti tehtäviä.

In [5]:
df['recorded_at'][df['subjectId'] == 30].unique()

array(['2022-05-26 09:38:57', '2022-05-26 10:14:22',
       '2022-05-26 09:39:40', '2022-05-26 10:14:05'], dtype=object)

10:14:05 alkaen. Vaihdetaan subjectId sen kellon ajan jälkeen.

In [6]:
df.loc[(df['recorded_at'] == '2022-05-26 10:14:05') | (df['recorded_at'] == '2022-05-26 10:14:22'),'subjectId'] = 31

print('Osallistujia yhteensä: ', df['subjectId'].unique().size)
print('SubjectId 30 tehtävän suoritusajat: ', df['recorded_at'][df['subjectId'] == 30].unique())
print('SubjectId 31 tehtävän suoritusajat: ',df['recorded_at'][df['subjectId'] == 31].unique())
print('\nSubjectId = 29')
print(df['recall'][df['subjectId'] == 29].info())
print('\nSubjectId = 30')
print(df['recall'][df['subjectId'] == 30].info())
print('\nSubjectId = 31')
print(df['recall'][df['subjectId'] == 31].info())

Osallistujia yhteensä:  29
SubjectId 30 tehtävän suoritusajat:  ['2022-05-26 09:38:57' '2022-05-26 09:39:40']
SubjectId 31 tehtävän suoritusajat:  ['2022-05-26 10:14:22' '2022-05-26 10:14:05']

SubjectId = 29
<class 'pandas.core.series.Series'>
Int64Index: 212 entries, 3710 to 5299
Series name: recall
Non-Null Count  Dtype 
--------------  ----- 
12 non-null     object
dtypes: object(1)
memory usage: 3.3+ KB
None

SubjectId = 30
<class 'pandas.core.series.Series'>
Int64Index: 212 entries, 5300 to 5829
Series name: recall
Non-Null Count  Dtype 
--------------  ----- 
12 non-null     object
dtypes: object(1)
memory usage: 3.3+ KB
None

SubjectId = 31
<class 'pandas.core.series.Series'>
Int64Index: 212 entries, 5512 to 5935
Series name: recall
Non-Null Count  Dtype 
--------------  ----- 
12 non-null     object
dtypes: object(1)
memory usage: 3.3+ KB
None


## Poistetaan subjectId 1 pois Ospan aineistosta
Tiputetaan subjectId 1 pois datasetistä.

In [7]:
df = df[df.subjectId != 1]
print('Osallistujia yhteensä: ', df['subjectId'].unique().size)

Osallistujia yhteensä:  28


Nyt kaikilla riveillä on oikea subjectId ja datasetissä oikeat osallistujat.

## Ospan datasetin tutkiminen

Tutkitaan ensin koko datasettiä ja sen sisältämää tietoa.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5936 entries, 106 to 6147
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   success              112 non-null    object 
 1   trial_type           5936 non-null   object 
 2   trial_index          5936 non-null   int64  
 3   time_elapsed         5936 non-null   int64  
 4   internal_node_id     5936 non-null   object 
 5   run_id               5936 non-null   int64  
 6   condition            5936 non-null   int64  
 7   recorded_at          5936 non-null   object 
 8   source_code_version  5936 non-null   object 
 9   ip                   0 non-null      float64
 10  user_agent           5936 non-null   object 
 11  device               5936 non-null   object 
 12  browser              5936 non-null   object 
 13  browser_version      5936 non-null   object 
 14  platform             5936 non-null   object 
 15  platform_version     5936 non-null  

Analyysia varten tarvitsemme seuraavat tiedot:
1. Osallistujan ID (SubjectId)
2. Kummassa koetilanteessa tehtävä on suoritettu (cond)
3. Kumpi tehtävistä suoritettiin ensin (bal)
4. Kuinka paljon osallistuja muisti kirjaimia oikein (accuracy)
5. Kuinka paljon osallistuja vastasi laskutehtäviin oikein (accuracy)

Pisteytyksen voisi myös halutessaan tarkistaa pisteyttämällä tulokset itse recall ja stimuli muuttujien avulla.

Tutkitaan missä halutut tiedot datassa sijaitsevat.

In [9]:
df.head(30)

,success,trial_type,trial_index,time_elapsed,internal_node_id,run_id,condition,recorded_at,source_code_version,ip,...,cond,bal,view_history,rt,stimulus,key_press,accuracy,recall,stimuli,set_size
106,True,fullscreen,0,993470,0.0-0.0,2,1,2022-05-05 14:02:39,07e2e056772a877f89175bcd11545e20,NaN,...,2,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,NaN,instructions,1,996580,0.0-1.0,2,1,2022-05-05 14:02:39,07e2e056772a877f89175bcd11545e20,NaN,...,2,a,"[{""page_index"":0,""viewing_time"":3108}]",3108.0,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,html-keyboard-response,2,997583,0.0-2.0-0.0-0.0,2,1,2022-05-05 14:02:39,07e2e056772a877f89175bcd11545e20,NaN,...,2,a,NaN,NaN,"<div style=""font-size:70px;"">L<br><br><br><br>...",NaN,NaN,NaN,NaN,NaN
109,NaN,html-button-operationspan,3,1000436,0.0-2.0-0.0-1.0,2,1,2022-05-05 14:02:39,07e2e056772a877f89175bcd11545e20,NaN,...,2,a,NaN,2850.0,"<div style=""font-size:46px;"">9 - 1 = 6<br><br>...",NaN,1.0,NaN,NaN,NaN
110,NaN,html-keyboard-response,4,1000441,0.0-2.0-0.0-2.0,2,1,2022-05-05 14:02:39,07e2e056772a877f89175bcd11545e20,NaN,...,2,a,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
111,NaN,html-keyboard-response,5,1001457,0.0-2.0-0.0-0.1,2,1,2022-05-05 14:02:39,07e2e056772a877f89175bcd11545e20,NaN,...,2,a,NaN,NaN,"<div style=""font-size:70px;"">R<br><br><br><br>...",NaN,NaN,NaN,NaN,NaN
112,NaN,html-button-operationspan,6,1004523,0.0-2.0-0.0-1.1,2,1,2022-05-05 14:02:39,07e2e056772a877f89175bcd11545e20,NaN,...,2,a,NaN,3063.0,"<div style=""font-size:46px;"">3 - 1 = 2<br><br>...",NaN,1.0,NaN,NaN,NaN
113,NaN,html-keyboard-response,7,1004526,0.0-2.0-0.0-2.1,2,1,2022-05-05 14:02:39,07e2e056772a877f89175bcd11545e20,NaN,...,2,a,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
114,NaN,html-keyboard-response,8,1005528,0.0-2.0-0.0-0.2,2,1,2022-05-05 14:02:39,07e2e056772a877f89175bcd11545e20,NaN,...,2,a,NaN,NaN,"<div style=""font-size:70px;"">S<br><br><br><br>...",NaN,NaN,NaN,NaN,NaN
115,NaN,html-button-operationspan,9,1007340,0.0-2.0-0.0-1.2,2,1,2022-05-05 14:02:39,07e2e056772a877f89175bcd11545e20,NaN,...,2,a,NaN,1810.0,"<div style=""font-size:46px;"">2 - 1 = 0<br><br>...",NaN,1.0,NaN,NaN,NaN


Muuttujat subjectId, cond ja bal ovat mukana kaikilla riveillä osoittamassa missä tilanteessa tehtävät ovat suoritettu. Analyysin kannalta kiinnostavat accuracy arvot löytyvät riveiltä, joissa trial_type on 'operation-span-recall'. Niiden avulla saa laskettua kuinka paljon osallistuja muisti kirjaimia oikein. 

Lisäksi trial_type 'html-button-operationspan' sijaitsevista accuracy muuttujista täytyy laskea kuinka paljon osallistuja sai oikein laskutehtävistä. Niiden osallistujien, jotka saivat lasku tehtävistä alle 85% oikein, dataa ei voi käyttää analyysissa, koska heidän ei voi katsoa suorittaneen tehtävää oikein.

## Laskutehtävien suorittamisprosenttien laskenta
Valitaan vain trial_type 'html-button-operationspan' ja lasketaan oikein suoritettujen laskutehtävien prosenttiosuus.

In [10]:
df_math = df.loc[lambda df: df['trial_type'] == 'html-button-operationspan']

df_math = df_math[['subjectId', 'accuracy','cond']]

df_math.head()

,subjectId,accuracy,cond
109,3,1.0,2
112,3,1.0,2
115,3,1.0,2
118,3,1.0,2
121,3,1.0,2


In [11]:
print('Uniikit: ', df_math['subjectId'].unique())
print('Yhteensä: ',df_math['subjectId'].unique().size)
print('\nInfo:')
print(df_math.info())

Uniikit:  [ 3 10 11  8 12 14 13  9  2 15 16  6 23 22 17 21  4 18 29 19 20 28 25 27
 26 30  5 31]
Yhteensä:  28

Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1680 entries, 109 to 6142
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   subjectId  1680 non-null   int64  
 1   accuracy   1674 non-null   float64
 2   cond       1680 non-null   int64  
dtypes: float64(1), int64(2)
memory usage: 52.5 KB
None


Osalla riveistä muuttujassa accuracy puuttuu arvo.

In [12]:
df_math.loc[pd.isna(df_math["accuracy"]), :]

,subjectId,accuracy,cond
1246,13,NaN,2
2051,16,NaN,1
3174,22,NaN,1
4225,28,NaN,1
5391,30,NaN,1
5856,31,NaN,1


In [13]:
df.loc[1246,:]

success                                                              NaN
trial_type                                     html-button-operationspan
trial_index                                                           80
time_elapsed                                                     1938995
internal_node_id                                        0.0-2.0-0.4-1.23
run_id                                                                25
condition                                                              1
recorded_at                                          2022-05-11 12:54:10
source_code_version                     07e2e056772a877f89175bcd11545e20
ip                                                                   NaN
user_agent             Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
device                                                            WebKit
browser                                                             Edge
browser_version                                    

Näissä tapauksissa osallistuja ei ole ehtinyt vastata 6 sekunnin sisällä laskutehtävään, eli vastaus on mennyt väärin. Puuttuvat arvot tulee siis korvata arvolla 0.

In [14]:
df_math['accuracy'] = df_math['accuracy'].fillna(0)

Lasketaan laskutehtävässä onnistumisprosentti molemmissa koetilanteissa erikeen sekä yhteenlaskettuna. Taulukossa näkyy kaikki rivit joilla tarkkuus on alle 85%

In [15]:
print('85% oikein vastauksia: >',30*0.85)
df_sum = df_math.groupby(by=['subjectId','cond']).sum()
df_sum[df_sum['accuracy'] < 26]

85% oikein vastauksia: > 25.5


,,accuracy
subjectId,cond,
3,1,25.0


In [16]:
print('85% oikein vastauksia: >',60*0.85)
df_sum2 = df_math.groupby(by=['subjectId']).sum()
df_sum2[df_sum2['accuracy'] < 52]

85% oikein vastauksia: > 51.0


,accuracy,cond
subjectId,,


In [17]:
print('25 oikein on', round(25/30*100, 1), '% oikein.')
df_sum.loc[[3]]

25 oikein on 83.3 % oikein.


accuracy
subjectId cond          
3         1         25.0
          2         27.0

Osallistujalla 3 koetilanteessa 1 tarkkuus jää juuri alle rajaarvon. Molemmat koetilanteet huomioon ottaen onnistumisprosentti juuri ylittää rajaarvon.

Pidetään kaikki osallistujat mukana?

## Ospan datan järjestely
Kertauksena, analyysia varten tarvitsemme seuraavat tiedot:
1. Osallistujan ID (SubjectId)
2. Kummassa koetilanteessa tehtävä on suoritettu (cond)
3. Kumpi tehtävistä suoritettiin ensin (bal)
4. Kuinka paljon osallistuja muisti kirjaimia oikein (accuracy)
    - Tästä tulee laskea summa

In [18]:
df_in = df.loc[lambda df: df['trial_type'] == 'operation-span-recall']

df_ospan = df_in[['subjectId','cond','accuracy','bal']]
df_ospan

,subjectId,cond,accuracy,bal
126,3,2,3.0,a
140,3,2,3.0,a
154,3,2,4.0,a
171,3,2,3.0,a
191,3,2,3.0,a
...,...,...,...,...
6070,5,2,4.0,a
6090,5,2,2.0,a
6107,5,2,5.0,a
6127,5,2,1.0,a


Jaetaan koetilanteiden pisteet erillisiin dataframeihin ja yhdistetään ne ja counterbalanssi muuttuja

In [19]:
cond1 = df_ospan[df_ospan['cond'] == 1]
cond1 = cond1.drop(['cond'], axis=1)
cond1 = cond1.rename(columns = {'accuracy': 'score_1'})
cond1 = cond1.reset_index().drop('index',axis=1)
cond1 = cond1.groupby(by=['subjectId']).sum()

cond2 = df_ospan[df_ospan['cond'] == 2]
cond2 = cond2.drop(['cond'], axis=1)
cond2 = cond2.rename(columns = {'accuracy': 'score_2'})
cond2 = cond2.set_index('subjectId').sort_index()
cond2 = cond2.groupby(by=['subjectId']).sum()

df_join = cond1.join(cond2)

df_bal = df_in[['subjectId','bal']]
df_bal = df_bal.drop_duplicates().set_index('subjectId').sort_index()

df_join = df_join.join(df_bal)
df_join

,score_1,score_2,bal
subjectId,,,
2,19.0,25.0,b
3,18.0,21.0,a
4,26.0,26.0,b
5,22.0,21.0,a
6,28.0,30.0,b
8,30.0,28.0,b
9,30.0,26.0,a
10,30.0,30.0,b
11,25.0,30.0,a


Ospan tulokset on nyt järjestettynä

### Pisteiden datatyypin vaihdos
Pisteet ovat aina kokonaislukuja, joten poistetaan decimaalit.

In [20]:
df_join['score_1'] = df_join['score_1'].astype('int64')
df_join['score_2'] = df_join['score_2'].astype('int64')

# Kyselyn aineisto

Kyselyllä hankittu data tulee yhdistää ospan tuloksiin. Webropolista tuotuna columnien nimet ovat mahdottoman pitkiä, minkä takia columnien nimet on lyhennetty excelillä. 

## Datan tarkistus
Tarkistetaan ensin, että aineisto ei ole muuttunut koodauksen aikana.

In [21]:
df_survey_coded=pd.read_csv('Raw data/Ennakko-gradu_Perusraportti_koodattu.csv', delimiter=';' )
df_survey_raw=pd.read_csv('Raw data/Ennakko-gradu_Perusraportti.csv', delimiter=';' )
df_survey_coded[np.all(df_survey_coded.values == df_survey_raw.values,1)].any().any()

True

Ei ole muuttunut.

## Dataframen muuttujan nimi lyhyempään muotoon

In [22]:
df_survey = df_survey_coded

## Poistetaan osallistujanumero 1

In [23]:
df_survey = df_survey[df_survey.subjectId != 1]
print('Osallistujia yhteensä: ', df_survey['subjectId'].unique().size)

Osallistujia yhteensä:  28


## Suostumus tutkimukseen osallistumisesta
Tarkistetaan, että jokainen osallistuja on antanut suostumuksen tutkimukseen osallistumiselle.

In [24]:
df_survey['consent'].count()

28

Jokainen osallistuja on antanut suostumuksen. Se rivi voidaan poistaa aineistosta.

In [25]:
df_survey = df_survey.drop('consent',axis=1)

## Ylimääräisen sarakkeen poistaminen

Saraketta 'result' ei tarvita analyysissa.

In [26]:
df_survey = df_survey.drop('result',axis=1)

## Yhdistetään kyselyainesto ospan-aineistoon

In [27]:
df_survey = df_survey.set_index('subjectId').sort_index()

df_survey.head()

,gender,age,vdaq_1,vdaq_2,vdaq_3,vdaq_4,vdaq_5,vdaq_6,vdaq_7,vdaq_8,...,ovs_3,ovs_4,ovs_5,ovs_6,ovs_7,ovs_8,ovs_9,ovs_10,ovs_11,ovs_12
subjectId,,,,,,,,,,,,,,,,,,,,,
2,1,31,3,3,3,4,4,3,3,3,...,2,2.0,3,2,3,2,3,2,2,2
3,1,26,3,4,4,4,4,4,4,2,...,2,2.0,5,5,5,2,2,4,5,3
4,1,28,3,1,4,4,2,3,2,3,...,1,2.0,4,4,2,2,4,5,5,3
5,2,26,3,4,2,3,4,2,3,2,...,3,4.0,4,4,4,4,2,2,2,2
6,2,25,3,2,4,2,2,3,3,3,...,2,2.0,3,4,3,2,2,4,4,3


In [28]:
df_combined = df_join.join(df_survey)
df_combined = df_combined.reset_index()

df_combined.head()

,subjectId,score_1,score_2,bal,gender,age,vdaq_1,vdaq_2,vdaq_3,vdaq_4,...,ovs_3,ovs_4,ovs_5,ovs_6,ovs_7,ovs_8,ovs_9,ovs_10,ovs_11,ovs_12
0,2,19,25,b,1,31,3,3,3,4,...,2,2.0,3,2,3,2,3,2,2,2
1,3,18,21,a,1,26,3,4,4,4,...,2,2.0,5,5,5,2,2,4,5,3
2,4,26,26,b,1,28,3,1,4,4,...,1,2.0,4,4,2,2,4,5,5,3
3,5,22,21,a,2,26,3,4,2,3,...,3,4.0,4,4,4,4,2,2,2,2
4,6,28,30,b,2,25,3,2,4,2,...,2,2.0,3,4,3,2,2,4,4,3


## Muutetaan `0` arvot `NaN`
"En osaa vastata" vastaukset tulee tulkita tyhjäksi, joten kaikki `0` arvot tulee muuttaa `NaN`

In [29]:
df_combined = df_combined.where(df_combined != 0, np.nan)

## Arvojen kääntäminen
Osa kysymyksistä on ollut käännettyjä, joten niiden kysymysten arvot täytyy muuttaa.

In [30]:
df_combined[['asq_2','asq_3','asq_7','asq_10', 'asq_12']] = df_combined[['asq_2','asq_3','asq_7','asq_10', 'asq_12']].replace([1,2,3,4,5,6], [6,5,4,3,2,1])

## Summamuuttujien luonti
Kyselyn vastauksista tulee luoda summamuuttujat. Osallistujat ovat voineet myös jättää vastaamatta niin halutessaan, mikä tekee summamuuttujan luomisesta hieman monimutkaisempaa. Jos kysymykseen ei ole vastattu, sitä ei pidä ottaa huomioon keskiarvon laskennassa. Toisinsanoen arvot `NaN` tulee jättää huomioimatta. 

pandas laskee keskiarvon oletuksena oikein, jos `0` tilalla on `NaN`

In [31]:
df_mims = df_combined

df_mims['vdaq'] = df_mims[['vdaq_1', 'vdaq_2','vdaq_3','vdaq_4','vdaq_5','vdaq_6','vdaq_7','vdaq_8','vdaq_9','vdaq_10','vdaq_11','vdaq_12','vdaq_13','vdaq_14','vdaq_15','vdaq_16',]].mean(axis=1)

df_mims['asq_int'] = df_mims[['asq_2', 'asq_4', 'asq_5', 'asq_6', 'asq_7', 'asq_8', 'asq_9']].mean(axis=1)
df_mims['asq_ext'] = df_mims[['asq_1', 'asq_3', 'asq_10', 'asq_11', 'asq_12']].mean(axis=1)
df_mims['asq'] = df_mims[['asq_1', 'asq_2' ,'asq_3', 'asq_4', 'asq_5', 'asq_6', 'asq_7','asq_8','asq_9', 'asq_10', 'asq_11', 'asq_12']].mean(axis=1)

df_mims['ovs'] = df_mims[['ovs_1', 'ovs_2', 'ovs_3', 'ovs_4', 'ovs_5', 'ovs_6', 'ovs_7', 'ovs_8', 'ovs_9', 'ovs_10', 'ovs_11', 'ovs_12']].mean(axis=1)
df_mims['ovs_salience'] = df_mims[['ovs_1', 'ovs_2', 'ovs_3', 'ovs_4']].mean(axis=1)
df_mims['ovs_reactibility'] = df_mims[['ovs_5', 'ovs_6', 'ovs_7', 'ovs_8']].mean(axis=1)
df_mims['ovs_monitoring'] = df_mims[['ovs_9', 'ovs_10', 'ovs_11', 'ovs_12']].mean(axis=1)

# Tiedoston tallennus

In [32]:
# Pois kommentista, jos halutaan todella kirjoittaa uusi tiedosto
df_mims.to_csv('ospan_and_survey.csv',index=False)